# Convert raw txt output to JSON

Columns in txt are:

```bifurcation ID, number of bounces, pr, ptheta, pphi, energy, rocking number, in phase/out of phase, period```

In [67]:
import math

# Returns a set of all prime factors of n
def primeFactors(n):
    if (n == 0):
        return set([])
    factors = []

    # Print the number of twos that divide n 
    while n % 2 == 0:
        factors.append(2)
        n = n // 2
          
    # n must be odd at this point 
    # so a skip of 2 (i = i + 2) can be used 
    for i in range(3, int(math.sqrt(n))+1, 2): 
        # while i divides n, print i and divide n
        while n % i == 0:
            factors.append(i)
            n = n // i
              
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        factors.append(n)
    
    return set(factors)

# Returns a set of all factors of n
def factors(n):
    if (n == 0):
        return set([])
    
    factors = []
    for i in range(1, n//2+1):
        if n % i == 0:
            factors.append(i)
    return set(factors)

print(primeFactors(0))
print(factors(98))

set()
{1, 2, 7, 14, 49}


In [40]:
a = {3:2, 5:4}
print(5 in a)

True


# Test code for something else

In [ ]:
class BifLoc:
    def __init__(self, E, m):
        self.E = E
        self.m = m

# Does a binary search to find where E belongs in the bifurcation locations.
# If the region is surrounded by two found bifurcations such that the
# rocking numbers are next to each other, then the region is empty.
# m_max is the maximum rocking number we're going to allow.
def is_empty(E, bif_locs, m_max):
    if E == 0:
        return True
    i = 0
    j = len(bif_locs)
    while j-i > 1:
        k = (i+j)//2
        if bif_locs[k].E <= E:
            i = k
        else:
            j = k
    if bif_locs[i].m == bif_locs[j].m-1 or bif_locs[i].m == m_max:
        return True
    return False

bif_locs = [BifLoc(-1/3, 0), BifLoc(-.22, 3), BifLoc(-.1, 4), BifLoc(0, math.inf)]
bif_locs[-1].E
print(is_empty(-.2, bif_locs, 9) == True)
print(is_empty(-.23, bif_locs, 9) == False)
print(is_empty(-.22, bif_locs, 9) == True)
print(is_empty(-.04, bif_locs, 9) == False)
print(is_empty(-1/3, bif_locs, 9) == False)
print(is_empty(0, bif_locs, 9) == True)

print(is_empty(-.2, bif_locs, 4) == True)
print(is_empty(-.23, bif_locs, 4) == False)
print(is_empty(-.22, bif_locs, 4) == True)
print(is_empty(-.04, bif_locs, 4) == True)
print(is_empty(-1/3, bif_locs, 4) == False)
print(is_empty(0, bif_locs, 4) == True)

print(is_empty(-.2, bif_locs, 5) == True)
print(is_empty(-.23, bif_locs, 5) == False)
print(is_empty(-.22, bif_locs, 5) == True)
print(is_empty(-.04, bif_locs, 5) == False)
print(is_empty(-1/3, bif_locs, 5) == False)
print(is_empty(0, bif_locs, 5) == True)


In [125]:
import queue 
  
# From class queue, Queue is 
# created as an object Now L 
# is Queue of a maximum  
# capacity of 20 
L = queue.Queue() 
  
# Data is inserted into Queue 
# using put() Data is inserted 
# at the end 
L.put(5) 
L.put(9) 
L.put(1) 
L.put(7) 
  
# get() takes data out from 
# the Queue from the head  
# of the Queue 
print(L.get()) 
print(L.get()) 
print(L.get()) 
print(L.get()) 

5
9
1
7


# Simple conversion not checking for prime factors

In [183]:
import json
import re
import os

def convert():
    data = []
    for num_bounces in range(1, 128):
        fn = './states{:03d}.txt'.format(num_bounces)
        exists = os. path. isfile(fn)
        if exists:
            with open(fn, 'r') as infile:
                print('reading {}'.format(fn))
                for line in infile:
                    line = line.strip()
                    (mode_id, n, pr, ptheta, pphi, E, ptheta_rocking, pphi_rocking, phase, period) = line.split(' ')
                    state = { 'id':int(mode_id), 'n':int(n), 'pr':float(pr),
                             'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                             'ptheta_rocking':int(ptheta_rocking), 'pphi_rocking':int(pphi_rocking),
                             'phase':int(phase), 'period':float(period) }
                    data.append(state)

    with open('./states_all.json', 'w') as outfile:  
        json.dump(data, outfile)

    print('done')

convert()
# %timeit convert()

reading ./states001.txt
reading ./states002.txt
reading ./states003.txt
reading ./states004.txt
reading ./states005.txt
reading ./states006.txt
reading ./states007.txt
reading ./states008.txt
reading ./states009.txt
reading ./states010.txt
reading ./states011.txt
done


# Check for factors

In [71]:
import json
import re

def convert():
    state_map = {}
#     pfactors = [primeFactors(i) for i in range(129)]
    pfactors = [factors(i) for i in range(129)]
    cur_mode = -1
    skip_mode = False
    data = []
    with open('./states.txt', 'r') as infile:
        for line in infile:
            line = line.strip()
            (mode_id, n, pr, ptheta, pphi, E, rocking, phase, period) = line.split(' ')
            mode_id = int(mode_id)
            n = int(n)
            E = float(E)
            rocking = int(rocking)
            state = { 'id':int(mode_id), 'n':int(n), 'pr':float(pr),
                         'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                         'rocking':int(rocking), 'phase':int(phase), 'period':float(period) }
            
            if mode_id != cur_mode:
                # Look to see if a prime factor of n already added this state. This functionality relies on
                # the text data coming to being sorted on n.
                unique = True
                for factor_n in pfactors[n]:
                    if (unique and (rocking//(n//factor_n), factor_n) in state_map):
                        # A prime factor bouncing number has a mode with this rocking number.
                        # Check to see if it is the same mode.
                        prev_state = state_map[(rocking//(n//factor_n), factor_n)]
                        if abs(prev_state['E'] - state['E']) < 0.001:
                            # The current state is redundant. Ignore.
                            unique = False
                            print('Filtering out ({},{},{},{}) from ({},{},{},{})'.format(
                                prev_state['n'], prev_state['rocking'], prev_state['E'], prev_state['pphi'],
                                state['n'], state['rocking'], state['E'], state['pphi']))
                if unique:
                    state_map[(rocking, n)] = state
                    data.append(state)
                    skip_mode = False
                else:
                    skip_mode = True
            elif not skip_mode:
                data.append(state)
            
            cur_mode = mode_id

    with open('./states_unique.json', 'w') as outfile:  
        json.dump(data, outfile)

    print('done')

convert()
# %timeit convert()

Filtering out (1,1,-0.06457,0.001) from (2,2,-0.06457,0.001)
Filtering out (1,1,-0.06457,0.001) from (3,3,-0.06457,0.001)
Filtering out (2,1,-0.1655,0.001) from (4,2,-0.1655,0.001)
Filtering out (1,1,-0.06457,0.001) from (4,4,-0.06457,0.001)
Filtering out (1,1,-0.06457,0.001) from (5,5,-0.06457,0.001)
Filtering out (3,1,-0.23888,0.001) from (6,2,-0.23888,0.001)
Filtering out (2,1,-0.1655,0.001) from (6,3,-0.1655,0.001)
Filtering out (3,2,-0.12651,0.001) from (6,4,-0.12651,0.001)
Filtering out (1,1,-0.06457,0.001) from (6,6,-0.06457,0.001)
Filtering out (4,1,-0.27253,0.001) from (8,2,-0.27253,0.001)
Filtering out (2,1,-0.1655,0.001) from (8,4,-0.1655,0.001)
Filtering out (4,3,-0.10847,0.001) from (8,6,-0.10847,0.001)
Filtering out (1,1,-0.06457,0.001) from (8,8,-0.06457,0.001)
Filtering out (4,9,-0.01326,0.001) from (8,18,-0.01326,0.001)
Filtering out (3,1,-0.23888,0.001) from (9,3,-0.23888,0.001)
Filtering out (3,2,-0.12651,0.001) from (9,6,-0.12651,0.001)
Filtering out (5,1,-0.29164,0

Filtering out (21,1,-0.33064,0.001) from (84,4,-0.33064,0.001)
Filtering out (14,1,-0.32734,0.001) from (84,6,-0.32734,0.001)
Filtering out (42,4,-0.32284,0.001) from (84,8,-0.32284,0.001)
Filtering out (42,5,-0.31723,0.001) from (84,10,-0.31723,0.001)
Filtering out (7,1,-0.31064,0.001) from (84,12,-0.31064,0.001)
Filtering out (28,1,-0.30417,0.001) from (84,3,-0.3032,0.001)
Filtering out (14,3,-0.28635,0.001) from (84,18,-0.28635,0.001)
Filtering out (42,13,-0.24851,0.001) from (84,26,-0.24851,0.001)
Filtering out (21,8,-0.22006,0.001) from (84,32,-0.22006,0.001)
Filtering out (28,3,-0.1655,0.001) from (84,9,-0.16475,0.001)
Filtering out (42,55,-0.04275,0.001) from (84,111,-0.04201,0.001)
Filtering out (17,1,-0.32925,0.001) from (85,5,-0.32925,0.001)
Filtering out (17,2,-0.31759,0.001) from (85,10,-0.31759,0.001)
Filtering out (17,6,-0.23103,0.001) from (85,30,-0.23103,0.001)
Filtering out (43,2,-0.33076,0.001) from (86,4,-0.33076,0.001)
Filtering out (43,4,-0.32331,0.001) from (86,8,

done
